In [7]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature


In [11]:
data=pd.read_csv('https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv',sep=';')
data


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [85]:
train,test=train_test_split(data,test_size=0.25,random_state=42)

In [86]:
train_x=train.drop(['quality'],axis=1).values
train_y=train['quality'].values.ravel()
test_x=test.drop(['quality'],axis=1).values
test_y=test['quality'].values.ravel()

train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,train_size=0.2,random_state=42)
signature=infer_signature(train_x,train_y)

In [87]:
### ANN Model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## Define model architecture
    mean=np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params wwith MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        ## Evaluate the model
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse=eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ## log the model

        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [88]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [89]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
}

In [90]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


Epoch 1/3                                            

12/12 [==============================] - 0s 15ms/step - loss: 26.8621 - root_mean_squared_error: 5.1829 - val_loss: 13.2394 - val_root_mean_squared_error: 3.6386

Epoch 2/3                                            

12/12 [==============================] - 0s 9ms/step - loss: 6.1099 - root_mean_squared_error: 2.4718 - val_loss: 3.3148 - val_root_mean_squared_error: 1.8206

Epoch 3/3                                            

12/12 [==============================] - 0s 8ms/step - loss: 4.4034 - root_mean_squared_error: 2.0984 - val_loss: 5.1930 - val_root_mean_squared_error: 2.2788

46/46 [==============================] - 0s 1ms/step - loss: 5.1930 - root_mean_squared_error: 2.2788

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]INFO:tensorflow:Assets written to: C:\Users\santh\AppData\Local\Temp\tmpxbg36uow\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\santh\AppData\Local\Temp\tmpxbg36uow\model\data\model\assets



Epoch 1/3                                                                     

12/12 [==============================] - 0s 23ms/step - loss: 38.5932 - root_mean_squared_error: 6.2123 - val_loss: 33.6852 - val_root_mean_squared_error: 5.8039

Epoch 2/3                                                                     

12/12 [==============================] - 0s 10ms/step - loss: 29.4469 - root_mean_squared_error: 5.4265 - val_loss: 25.3282 - val_root_mean_squared_error: 5.0327

Epoch 3/3                                                                     

12/12 [==============================] - 0s 9ms/step - loss: 22.1552 - root_mean_squared_error: 4.7069 - val_loss: 19.0526 - val_root_mean_squared_error: 4.3649

46/46 [==============================] - 0s 1ms/step - loss: 19.0526 - root_mean_squared_error: 4.3649

 25%|██▌       | 1/4 [00:05<00:14,  4.68s/trial, best loss: 2.278822660446167]INFO:tensorflow:Assets written to: C:\Users\santh\AppData\Local\Temp\tmp2kw3wsnq\model\dat

INFO:tensorflow:Assets written to: C:\Users\santh\AppData\Local\Temp\tmp2kw3wsnq\model\data\model\assets



Epoch 1/3                                                                     

12/12 [==============================] - 0s 14ms/step - loss: 7.9774 - root_mean_squared_error: 2.8244 - val_loss: 1.5648 - val_root_mean_squared_error: 1.2509

Epoch 2/3                                                                     

12/12 [==============================] - 0s 9ms/step - loss: 1.2003 - root_mean_squared_error: 1.0956 - val_loss: 1.1934 - val_root_mean_squared_error: 1.0924

Epoch 3/3                                                                     

12/12 [==============================] - 0s 9ms/step - loss: 0.9287 - root_mean_squared_error: 0.9637 - val_loss: 1.0186 - val_root_mean_squared_error: 1.0093

46/46 [==============================] - 0s 2ms/step - loss: 1.0186 - root_mean_squared_error: 1.0093

 50%|█████     | 2/4 [00:10<00:09,  4.81s/trial, best loss: 2.278822660446167]INFO:tensorflow:Assets written to: C:\Users\santh\AppData\Local\Temp\tmppb_s1l43\model\data\model\

...

Epoch 1/3                                                                      

12/12 [==============================] - 0s 16ms/step - loss: 14.2009 - root_mean_squared_error: 3.7684 - val_loss: 3.4951 - val_root_mean_squared_error: 1.8695

Epoch 2/3                                                                      

12/12 [==============================] - 0s 10ms/step - loss: 2.3075 - root_mean_squared_error: 1.5190 - val_loss: 1.9401 - val_root_mean_squared_error: 1.3929

Epoch 3/3                                                                      

12/12 [==============================] - 0s 14ms/step - loss: 1.5654 - root_mean_squared_error: 1.2512 - val_loss: 1.6579 - val_root_mean_squared_error: 1.2876

46/46 [==============================] - 0s 2ms/step - loss: 1.6579 - root_mean_squared_error: 1.2876

 75%|███████▌  | 3/4 [00:15<00:04,  4.91s/trial, best loss: 1.0092644691467285]INFO:tensorflow:Assets written to: C:\Users\santh\AppData\Local\Temp\tmpo0hexclt\model\data

INFO:tensorflow:Assets written to: C:\Users\santh\AppData\Local\Temp\tmpo0hexclt\model\data\model\assets



100%|██████████| 4/4 [00:19<00:00,  4.90s/trial, best loss: 1.0092644691467285]
INFO:tensorflow:Assets written to: C:\Users\santh\AppData\Local\Temp\tmpksjdr9ck\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\santh\AppData\Local\Temp\tmpksjdr9ck\model\data\model\assets



Best parameters: {'lr': 0.02986819170970855, 'momentum': 0.49894814987501745}
Best eval rmse: 1.0092644691467285
